# Creation of output files (combinaison of tas and snc files)

## Objective

The goal of this notebook is to create output files expressing snow cover as a function of warming level and no longer as a function of time, for all models.

In [1]:
from watermark import watermark
%load_ext watermark
print(watermark())

Last updated: 2025-07-08T11:46:02.678502+00:00

Python implementation: CPython
Python version       : 3.12.8
IPython version      : 8.17.2

Compiler    : GCC 13.3.0
OS          : Linux
Release     : 6.8.0-58-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit



In [2]:
import sys
import os

# Chemin absolu vers le dossier codes_ORL_evaluation
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Ajout du chemin au sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("Projet root ajouté au path:", project_root)

Projet root ajouté au path: /home/jovyan/M2_Internship_Laurie_Vayssettes


In [3]:
pip install cdo

  Using cached cdo-1.6.1-py3-none-any.whl.metadata (14 kB)
Using cached cdo-1.6.1-py3-none-any.whl (16 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
from module import *

/srv/conda/envs/notebook/lib/python3.12/site-packages/esmpy/interface/loadESMF.py:94: VersionWarning: ESMF installation version 8.8.0, ESMPy version 8.8.0b0
  warnings.warn("ESMF installation version {}, ESMPy version {}".format(


In [5]:
%watermark --iversions

xesmf     : 0.8.8
matplotlib: 3.10.0
numpy     : 2.0.2
skimage   : 0.25.0
cartopy   : 0.24.0
xarray    : 2025.1.1
seaborn   : 0.13.2
netCDF4   : 1.7.2
cdo       : 1.6.1
watermark : 2.5.0
pandas    : 2.2.3
sys       : 3.12.8 | packaged by conda-forge | (main, Dec  5 2024, 14:24:40) [GCC 13.3.0]
csv       : 1.0



In [11]:
scenario = 'ssp585' #chosen scenario
vsnow = 'snc' #snow variable
racineCMIP6="/home/jovyan/shared-storage/Data_LaurieV/input_data/"
racineMasques = "/home/jovyan/shared-storage/Data_LaurieV/input_data/mask/"

sncfois100 = "BCC-CSM2-MR BCC-ESM1 FGOALS-f3-L" #some models express in %, it is necessary to put it back in fraction

Yref1 = 1995 #reference period
Yref2 = 2014

limit2100 = True #limit to 2100

nmon = 12 

Tbinmin = -1.5 #min temperature
Tbinmax = 4. #max temperature
dtbin = .25 #threshold between temperature
nbin = np.int32((Tbinmax - Tbinmin)/dtbin + 1)
Tbincen = np.linspace(Tbinmin,Tbinmax,nbin)

taboo = [ '' ]

epsilon = 0.001

def get_models(filelist):
    models = []
    for ific,fichier in enumerate(filelist):
        model = fichier.rsplit("/",1)[1].split("_")[2]
        if (model not in models): 
            models.append(model)
    models.sort()
    return(models)
    
filelist = glob.glob("/home/jovyan/shared-storage/Data_LaurieV/input_data/historical-Amon-tas/*.nc")
models_Thist = get_models(filelist)

filelist = glob.glob("/home/jovyan/shared-storage/Data_LaurieV/input_data/historical-LImon-snc/*.nc")
models_Shist = get_models(filelist)

filelist = glob.glob("/home/jovyan/shared-storage/Data_LaurieV/input_data/ssp585-Amon-tas/*.nc")
models_Tscen = get_models(filelist)

filelist = glob.glob("/home/jovyan/shared-storage/Data_LaurieV/input_data/ssp585-LImon-snc/*.nc")
models_Sscen = get_models(filelist)

models = []
for imod, model in enumerate(models_Thist):
    if (model not in models) and (model not in taboo) and (model in models_Shist) and (model in models_Tscen) and (model in models_Sscen):
        models.append(model)

print(models)

for imod, model in enumerate(models):

    print("Model : {} {}".format(imod,model))

    # areacella
    #
    champ = 'areacella'
    fichier = glob.glob(racineMasques+'/'+champ+'/'+champ+'_'+model+'.nc')
    if (fichier == []):
        fichier = glob.glob(racineMasques+'/pseudo_'+champ+'/'+champ+'_'+model+'.nc')
    if (fichier == []):
        print("Il manque {} pour {} !".format(champ,model))
        quit()
    print(fichier)
    f = netCDF4.Dataset(fichier[0])
    lon = f.variables['lon'][:]
    nlon = lon.size
    print(nlon)
    lat = f.variables['lat'][:]
    nlat = lat.size
    print(nlat)
    areacella = f.variables[champ][:,:]
    aireterre = np.sum(areacella)
    # print("Aire Terre : {}".format(aireterre))
    f.close()

    # lire Thist + Tscen, extraire les temps, calculer les temperatures moyennes globales
    #
    champ = 'tas'
    fichier = glob.glob("/home/jovyan/shared-storage/Data_LaurieV/input_data/historical-Amon-tas/"+"/*_"+model+"_*.nc")
    print(fichier)
    fh = netCDF4.Dataset(fichier[0])
    fichier = glob.glob("/home/jovyan/shared-storage/Data_LaurieV/input_data/ssp585-Amon-tas/"+"/*_"+model+"_*.nc")
    print(fichier)
    fs = netCDF4.Dataset(fichier[0])
    nlon2 = fh.variables['lon'][:].size
    print(nlon2)
    nlat2 = fh.variables['lat'][:].size
    print(nlat2)
    if (nlon2 != nlon) or (nlat2 != nlat):
        print("{} : Nombre de longitudes ou latitude faux ! {} {} {} {}".format(model,nlon,nlon2,nlat,nlat2))
        quit()
    tname = fh.variables[champ].dimensions[0]
    timeh = fh.variables[tname][:]
    nth = timeh.shape[0]
    times = fs.variables[tname][:]
    nts = times.shape[0]
    if (model == "NorESM2-LM") or (model == "NorESM2-MM"):
       nts -= 1
    #
    nt = nth + nts
    year = np.empty((nt),np.int32)
    month = np.empty((nt),np.int32)
    GSAT = np.empty((nt),np.float32)
    #
    t_unit = fh.variables[tname].units
    t_cal = fh.variables[tname].calendar
    tvalue = netCDF4.num2date(timeh,units = t_unit,calendar = t_cal)
    for it in range(nth):
        year[it] = tvalue[it].year
        month[it] = tvalue[it].month
        tas = fh.variables[champ][it,:,:]
        tas *= areacella
        GSAT[it] = np.sum(tas[:,:])/aireterre
    #
    t_unit = fs.variables[tname].units
    t_cal = fs.variables[tname].calendar
    tvalue = netCDF4.num2date(times,units = t_unit,calendar = t_cal)
    if (model == "NorESM2-LM") or (model == "NorESM2-MM"):
        itoffset = 1
    else:
        itoffset = 0
    for it in range(nts):
        year[nth+it] = tvalue[it+itoffset].year
        month[nth+it] = tvalue[it+itoffset].month
        tas = fs.variables[champ][it+itoffset,:,:]
        tas *= areacella
        GSAT[nth+it] = np.sum(tas[:,:])/aireterre
    #
    fh.close()
    fs.close()
    #
    iyref1 = np.argmax(year >= Yref1)
    iyref2 = nt - np.argmax(year[::-1] <= Yref2)
    GSATref = np.average(GSAT[iyref1:iyref2])
    print("  GSAT reference : {}".format(GSATref))
    GSAT[:] -= GSATref
    #
    tau = 60
    GSATsmooth = np.empty((nt),np.float32)
    GSATsmooth[0] = np.average(GSAT[0:tau])
    for it in range(1,nt):
        GSATsmooth[it] = 1./tau * GSAT[it] + (1. - 1./tau) * GSATsmooth[it-1]

    # lire Shist + Sscen
    #
    fichier = glob.glob("/home/jovyan/shared-storage/Data_LaurieV/input_data/historical-LImon-snc/"+"/*_"+model+"_*.nc")
    fh = netCDF4.Dataset(fichier[0])
    fichier = glob.glob("/home/jovyan/shared-storage/Data_LaurieV/input_data/ssp585-LImon-snc/"+"/*_"+model+"_*.nc")
    fs = netCDF4.Dataset(fichier[0])
    # la grille peut etre differente de celle de T
    lon = fh.variables['lon'][:]
    nlon = lon.size
    lat = fh.variables['lat'][:]
    nlat = lat.size
    #
    n_inbin = np.zeros((nbin,nmon),np.int32)
    snowbin = np.zeros((nbin,nmon,nlat,nlon),np.float32) 
    #
    snowread = np.empty((nlat,nlon),np.float32)
    spval = fh.variables[vsnow].missing_value
    #
    if (limit2100):
        ntr = nth + min((2100-2014)*12,nts)
    else:
        ntr = nt
    #
    for it in range(ntr):
        if (it < nth):
            snowread = fh.variables[vsnow][it,:,:]
        else:
            snowread = fs.variables[vsnow][it-nth,:,:]
        ibin = np.argmin(np.abs(GSATsmooth[it]-Tbincen[:]))
        imon = month[it] - 1
        snowbin[ibin,imon,:,:] += snowread[:,:]
        n_inbin[ibin,imon] += 1
    #
    units = fh.variables[vsnow].units
    stdname = fh.variables[vsnow].standard_name
    longname = fh.variables[vsnow].long_name
    #
    fh.close()
    fs.close()

    # normaliser les bins
    #
    for ibin in range(nbin):
        for imon in range(nmon):
            if (n_inbin[ibin,imon] > 0):
                snowbin[ibin,imon,:,:] /= n_inbin[ibin,imon]
            else:
                snowbin[ibin,imon,:,:] = spval
    if (model in sncfois100) and (vsnow == 'snc'):
         snowbin = np.where( np.abs(snowbin/spval -1.) > epsilon, snowbin*100, snowbin )

    # pour graphique warming level
    levels = [ 10., 25., 50., 75., 90.]
    nlev = len(levels)

    swl = np.empty((nlev,nmon,nlat,nlon),np.float32)
    for ilev,level in enumerate(levels):
        xgtl = np.where( (snowbin > level) & (np.abs(snowbin/spval -1.) > epsilon), dtbin, 0)
        swl[ilev,:,:,:] = np.sum(xgtl, axis = 0)
        swl[ilev,:,:,:] = np.where( (np.abs(snowbin[np.int32(nbin/2),:,:,:]/spval -1.) > epsilon), swl[ilev,:,:,:], spval) + Tbinmin

    # ecrire
    #
    f = netCDF4.Dataset('/home/jovyan/shared-storage/Data_LaurieV/output_data2/SnowWarming_1995_2014/SnowWarming_'+model+'_historical+'+scenario+'.nc',mode='w',format='NETCDF4_CLASSIC')
    #
    monname = 'month'
    month_dim = f.createDimension(monname, nmon)
    months = f.createVariable(monname, np.float32, (monname,))
    months.units = 'months'
    months.long_name = 'month'
    months.standard_name = 'month'
    months.axis = 'T'
    #
    binname = 'GSAT'
    tbin_dim = f.createDimension(binname, nbin)
    tbins = f.createVariable(binname, np.float32, (binname,))
    tbins.units = 'K'
    tbins.long_name = 'GSAT wrt. 1995-2014'
    tbins.standard_name = 'GSAT'
    tbins.axis = 'Z'
    #
    latname = 'lat'
    lat_dim = f.createDimension(latname, nlat)
    lats = f.createVariable(latname, np.float32, (latname,))
    lats.units = 'degrees_north'
    lats.long_name = 'latitude'
    lats.standard_name = 'latitude'
    lats.axis = 'Y'
    #
    lonname = 'lon'
    lon_dim = f.createDimension(lonname, nlon)
    lons = f.createVariable(lonname, np.float32, (lonname,))
    lons.units = 'degrees_east'
    lons.long_name = 'longitude'
    lons.standard_name = 'longitude'
    lons.axis = 'X'
    #
    levelname = 'Level'
    lev_dim = f.createDimension(levelname, nlev)
    levs = f.createVariable(levelname, np.float32, (levelname,))
    levs.units = '%'
    levs.long_name = 'Level'
    levs.standard_name = 'Level'
    levs.axis = 'Z'
    #
    snowbins = f.createVariable(vsnow+'bin',np.float32,(binname,monname,latname,lonname))
    snowbins.missing_value = spval
    snowbins.units = units
    snowbins.standard_name = stdname
    snowbins.long_name = longname
    #
    swls = f.createVariable('Limit',np.float32,(levelname,monname,latname,lonname))
    swls.missing_value = spval
    swls.units = 'K'
    swls.standard_name = 'Limit'
    swls.long_name = 'Limit'
    #
    months[:] = month[:nmon]
    tbins[:] = Tbincen[:]
    lats[:] = lat[:]
    lons[:] = lon[:]
    levs[:] = levels[:]
    snowbins[:,:,:,:] = snowbin[:,:,:,:]
    swls[:,:,:,:] = swl[:,:,:,:]
    #
    f.close()


['BCC-CSM2-MR', 'CESM2', 'CESM2-WACCM', 'CIESM', 'CNRM-CM6-1', 'CNRM-CM6-1-HR', 'CNRM-ESM2-1', 'CanESM5', 'CanESM5-CanOE', 'EC-Earth3', 'EC-Earth3-Veg', 'FGOALS-f3-L', 'FGOALS-g3', 'GFDL-CM4', 'GISS-E2-1-G', 'HadGEM3-GC31-LL', 'IPSL-CM6A-LR', 'MIROC-ES2L', 'MIROC6', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'NorESM2-LM', 'NorESM2-MM', 'UKESM1-0-LL']
Model : 0 BCC-CSM2-MR
['/home/jovyan/shared-storage/Data_LaurieV/input_data/mask//pseudo_areacella/areacella_BCC-CSM2-MR.nc']
320
160
['/home/jovyan/shared-storage/Data_LaurieV/input_data/historical-Amon-tas/tas_Amon_BCC-CSM2-MR_historical_r1i1p1f1_gn_185001-201412.nc']
['/home/jovyan/shared-storage/Data_LaurieV/input_data/ssp585-Amon-tas/tas_Amon_BCC-CSM2-MR_ssp585_r1i1p1f1_gn_201501-210012.nc']
320
160
  GSAT reference : 288.5494689941406
Model : 1 CESM2
['/home/jovyan/shared-storage/Data_LaurieV/input_data/mask//areacella/areacella_CESM2.nc']
288
192
['/home/jovyan/shared-storage/Data_LaurieV/input_data/historical-Amon-tas/tas_Amon

/tmp/ipykernel_1215/2859020322.py:74: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  areacella = f.variables[champ][:,:]
/tmp/ipykernel_1215/2859020322.py:114: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  tas = fh.variables[champ][it,:,:]
/tmp/ipykernel_1215/2859020322.py:128: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  tas = fs.variables[champ][it+itoffset,:,:]


  GSAT reference : 288.0142517089844


/tmp/ipykernel_1215/2859020322.py:172: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  snowread = fh.variables[vsnow][it,:,:]
/tmp/ipykernel_1215/2859020322.py:174: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  snowread = fs.variables[vsnow][it-nth,:,:]
/tmp/ipykernel_1215/2859020322.py:253: UserWarning: WARNING: missing_value cannot be safely cast to variable dtype
  snowbins.missing_value = spval
/tmp/ipykernel_1215/2859020322.py:259: UserWarning: WARNING: missing_value cannot be safely cast to variable dtype
  swls.missing_value = spval


Model : 2 CESM2-WACCM
['/home/jovyan/shared-storage/Data_LaurieV/input_data/mask//areacella/areacella_CESM2-WACCM.nc']
288
192
['/home/jovyan/shared-storage/Data_LaurieV/input_data/historical-Amon-tas/tas_Amon_CESM2-WACCM_historical_r1i1p1f1_gn_185001-201412.nc']
['/home/jovyan/shared-storage/Data_LaurieV/input_data/ssp585-Amon-tas/tas_Amon_CESM2-WACCM_ssp585_r1i1p1f1_gn_201501-210012.nc']
288
192
  GSAT reference : 287.8789978027344
Model : 3 CIESM
['/home/jovyan/shared-storage/Data_LaurieV/input_data/mask//pseudo_areacella/areacella_CIESM.nc']
288
192
['/home/jovyan/shared-storage/Data_LaurieV/input_data/historical-Amon-tas/tas_Amon_CIESM_historical_r1i1p1f1_gr_185001-201412.nc']
['/home/jovyan/shared-storage/Data_LaurieV/input_data/ssp585-Amon-tas/tas_Amon_CIESM_ssp585_r1i1p1f1_gr_201501-210012.nc']
288
192
  GSAT reference : 288.6036682128906
Model : 4 CNRM-CM6-1
['/home/jovyan/shared-storage/Data_LaurieV/input_data/mask//areacella/areacella_CNRM-CM6-1.nc']
256
128
['/home/jovyan/s

## Conclusion

This notebook has produced new files representing snow as a function of global warming. We now have files for 25 different models, which means that 25 models will be used to study the evolution of snow.